In [1]:
from ia_model.data_preparation import load_data, prepare_data, split_data
# from ia_model.model import  train_model, evaluate_model, save_model, build_dynamic_model
from ia_model.utils import ia_plot_history
from ia_model.model_final import grid_evaluate_combinations,objective, build_best_model
import joblib
import optuna
from tqdm.auto import tqdm

2024-03-22 16:23:23.487483: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 16:23:23.489932: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 16:23:23.521783: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 16:23:23.521801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 16:23:23.522697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Configuration parameters for the model and data preparation
FILE_PATH = 'aggregated_data4.csv' #This data can be download from https://rcavieses.pythonanywhere.com/
FEATURE_COL = ['NOMBRE PRINCIPAL','NOMBRE ESTADO','NOMBRE OFICINA','SST','PESO DESEMBARCADO_KILOGRAMOS']#Change this if you want to add input variables
TARGET_COL = 'PESO DESEMBARCADO_KILOGRAMOS'
CATEGORICAL_COL = ['NOMBRE PRINCIPAL','NOMBRE ESTADO','NOMBRE OFICINA']
NUMERIC_COL = ['PESO DESEMBARCADO_KILOGRAMOS','SST']
steps_forecast = 12
time_steps= 12 # number of steps that lstm forecast, change this will be errors on scoore metrics ;
# Example 
neurons_list = [264]
layers_list = [6]
activations_list = ['relu']  # List of activation functions to test, possible choice ['relu', 'sigmoid','tanh'] 
EPOCHS = 150
BATCH_SIZE = 24
MODEL_PATH = 'mi_modelo.h5'
SCALER_X_PATH = 'scaler_X.pkl'
SCALER_Y_PATH = 'scaler_Y.pkl'




In [3]:
# Load and prepare data
df = load_data(str(FILE_PATH))
X, y, scaler_x,scaler_y, encoder , n_features = prepare_data(df, TARGET_COL, time_steps,CATEGORICAL_COL,NUMERIC_COL)
X_train, X_test, y_train, y_test = split_data(X, y)

### grid search method

In [ ]:
# Build model and test different combination to find the best combination
best_configuration, best_mse, best_r2 = grid_evaluate_combinations(X_train, y_train, X_test, y_test, time_steps, n_features, neurons_list, layers_list, activations_list, epochs=50, batch_size=32)
print(f"Best configuration: {best_configuration}\nMSE: {best_mse}, R^2: {best_r2}")

### Bayesian Optimization Method

In [5]:
# Define hyperparameter ranges or choices
hyperparam_config = {
    'neurons': (128, 246),
    'layers': (1, 4),
    'activation': ['relu', 'linear'],
    'batch_size': [30, 36],
    'epochs': (24, 36)
}

In [6]:
# Define a callback function for Optuna that updates the progress bar
def update_progress_bar(study, trial):
    # Update the progress bar by one for each completed trial
    progress_bar.update(1)

# Initialize the progress bar
progress_bar = tqdm(total=25, desc='Optimization Progress', position=0)  # 'total' should match 'n_trials'

def wrapped_objective(trial):
    return objective(trial, X_train, y_train, X_test, y_test, time_steps, n_features, hyperparam_config)

# Initialize and run the study
study = optuna.create_study(direction='minimize')
study.optimize(wrapped_objective, n_trials=25)

# Print the best parameters and their corresponding value
print('Best trial:', study.best_trial.params)
print('Best trial R^2 score:', study.best_trial.user_attrs['R^2'])

Optimization Progress:   0%|          | 0/25 [00:00<?, ?it/s][I 2024-03-22 06:31:04,880] A new study created in memory with name: no-name-bff60f00-6597-4b3a-8ff6-cf9cb2a23c9a


1668/1668 [==============================] - 19s 11ms/step


[I 2024-03-22 07:30:43,839] Trial 0 finished with value: 2.959612250354024e-05 and parameters: {'neurons': 230, 'layers': 3, 'activation': 'linear', 'batch_size': 30, 'epochs': 27}. Best is trial 0 with value: 2.959612250354024e-05.


1668/1668 [==============================] - 10s 6ms/step


[I 2024-03-22 07:52:06,649] Trial 1 finished with value: 2.844286420833834e-05 and parameters: {'neurons': 243, 'layers': 2, 'activation': 'relu', 'batch_size': 36, 'epochs': 34}. Best is trial 1 with value: 2.844286420833834e-05.


...

1668/1668 [==============================] - 5s 3ms/step


[I 2024-03-22 14:27:52,395] Trial 12 finished with value: 2.523267225511718e-05 and parameters: {'neurons': 152, 'layers': 1, 'activation': 'relu', 'batch_size': 30, 'epochs': 30}. Best is trial 12 with value: 2.523267225511718e-05.


1668/1668 [==============================] - 12s 7ms/step


[I 2024-03-22 15:15:27,666] Trial 13 finished with value: 2.732379569680518e-05 and parameters: {'neurons': 152, 'layers': 3, 'activation': 'relu', 'batch_size': 30, 'epochs': 31}. Best is trial 12 with value: 2.523267225511718e-05.
[W 2024-03-22 15:31:10,089] Trial 14 failed with parameters: {'neurons': 157, 'layers': 1, 'activation': 'relu', 'batch_size': 30, 'epochs': 31} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/envs/cedo/lib/python3.11/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/tmp/ipykernel_9831/222967474.py", line 10, in wrapped_objective
    return objective(trial, X_train, y_train, X_test, y_test, time_steps, n_features, hyperparam_config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rcavieses/DLM-mexican-fish-catch-forecast/ia_model/mode

KeyboardInterrupt: 

In [10]:
study.best_trial

FrozenTrial(number=12, state=1, values=[2.523267225511718e-05], datetime_start=datetime.datetime(2024, 3, 22, 14, 9, 27, 689406), datetime_complete=datetime.datetime(2024, 3, 22, 14, 27, 52, 395664), params={'neurons': 152, 'layers': 1, 'activation': 'relu', 'batch_size': 30, 'epochs': 30}, user_attrs={'R^2': 0.270337980016056}, system_attrs={}, intermediate_values={}, distributions={'neurons': IntDistribution(high=246, log=False, low=128, step=1), 'layers': IntDistribution(high=4, log=False, low=1, step=1), 'activation': CategoricalDistribution(choices=('relu', 'linear')), 'batch_size': CategoricalDistribution(choices=(30, 36)), 'epochs': IntDistribution(high=36, log=False, low=24, step=1)}, trial_id=12, value=None)

### Results

params = {'neurons': 152, 'layers': 1, 'activation': 'relu', 'batch_size': 30, 'epochs': 30}
params2 = {'neurons': 174, 'layers': 4, 'activation': 'linear', 'batch_size': 30, 'epochs': 33}

In [7]:
# Build, Train, Evaluate the model using best parameter
best_configuration = {'layers': 3, 'neurons': 174, 'activation': 'relu'}  # gives r2 of 0.32
model, mse, r2 = build_best_model(X_train,y_train,X_test,y_test,time_steps, n_features, best_configuration, 33,30)


Epoch 1/33
5691/5691 [==============================] - 143s 25ms/step - loss: 3.0379e-05 - val_loss: 2.3586e-05
Epoch 2/33
5691/5691 [==============================] - 139s 24ms/step - loss: 2.7462e-05 - val_loss: 2.2723e-05
Epoch 3/33
5691/5691 [==============================] - 139s 24ms/step - loss: 2.6961e-05 - val_loss: 2.3192e-05
Epoch 4/33
5691/5691 [==============================] - 138s 24ms/step - loss: 2.7457e-05 - val_loss: 2.1274e-05
Epoch 5/33
5691/5691 [==============================] - 138s 24ms/step - loss: 2.7288e-05 - val_loss: 2.1146e-05
Epoch 6/33
5691/5691 [==============================] - 138s 24ms/step - loss: 2.6141e-05 - val_loss: 2.2236e-05
Epoch 7/33
5691/5691 [==============================] - 138s 24ms/step - loss: 2.5320e-05 - val_loss: 2.3601e-05
Epoch 8/33
5691/5691 [==============================] - 139s 24ms/step - loss: 2.6048e-05 - val_loss: 2.1546e-05
Epoch 9/33
5691/5691 [==============================] - 140s 25ms/step - loss: 2.5257e-05 - val_

In [ ]:
ia_plot_history(history)

In [ ]:
save_model(model,'mi_modelo.h5')
joblib.dump(scaler_x, 'scaler_X.pkl')  # Saves the scaler
joblib.dump(scaler_y, 'scaler_Y.pkl')  # Saves the scaler
joblib.dump(encoder, 'encoder_filename.pkl')  # Saves the encoder